In [8]:
%load_ext autoreload
%autoreload 2

from collections import defaultdict
from matplotlib import pyplot as plt

import networkx as nx

from src.data.graph import load_graph_data

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# 1. Data loading

In [9]:
graph_data = load_graph_data()
print("\nKeys:\n\t{}".format("    ".join(graph_data.keys())))

loading raw data from tsv files...
formatting articles...
formatting categories...
formatting links...
formatting paths...
formatting distance matrix...
building graph...

Keys:
	paths_finished    articles    links    shortest-path-distance-matrix    categories    paths_unfinished    graph


In [101]:
def get_edge_weights(dataframe: str) -> dict[tuple[str, str], int]:
    """
    Return a dictionary where the keys are tuples representing an edge (u, v) and values are the edges weights.
    The weight of an edge (u, v) is the number of times users went from u to v in their path.
    If a user clicked on '<', the article that was discarded does not contribute to the weight.
    """
    edge_weights: dict[tuple[str, str], int] = defaultdict(int)
    
    for _, row in graph_data[dataframe].iterrows():
        path = row['path']
        clean_path = []  # Path without '<'
        for element in path:
            if element == '<':
                edge_weights[(clean_path[-1], '<')] += 1
                clean_path.pop()
            else:
                clean_path.append(element)
                
        for i in range(1, len(clean_path)):
            source, dest = clean_path[i-1], clean_path[i]
            edge_weights[(source, dest)] += 1
    
    assert(sum(edge_weights.values()) == 
        sum(graph_data[dataframe]['path'].apply(len)) 
        - sum(graph_data[dataframe]['path'].apply(lambda l: l.count('<')))
        - len(graph_data[dataframe]))
    
    return edge_weights

   

In [102]:
def get_graph(dataframe: str) -> nx.DiGraph:
    """
    Generates a directed graph from the given dataframe (either 'paths_finished' or 'paths_unfinished').
    The nodes of the graph are the articles.
    The weight of the edges of the graph represents the number of times users went from one article to another.
    A special node '<' is created. 
    The weight of an edge from u to '<' represents the number of times users clicked on the back button after visiting u.
    """
    graph = nx.DiGraph()
    
    # Add nodes to graph
    graph.add_node('<')
    for _, node in graph_data['articles'].iterrows():
        graph.add_node(node['name'])
        
    # Add edges to graph
    edge_weights = get_edge_weights(dataframe)
    for (source, dest), count in edge_weights.items():
        graph.add_edge(source, dest, weight=count)
        
    assert(graph.number_of_nodes() == len(graph_data['articles']) + 1)
    assert(graph.number_of_edges() <= len(graph_data['links']))
    return graph

In [98]:
finished_graph = get_graph('paths_finished')